In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping

import tensorflow as tf

seed = 777
np.random.seed(seed)
tf.set_random_seed(seed)

Using TensorFlow backend.


In [2]:
df = pd.read_csv('e:/Train.csv', encoding= 'ansi')

In [3]:
x_row = df.iloc[:, :-1]
y_row = df.iloc[:, -1:]
x_row.head(3)

,총_유동인구_수,남성_유동인구_수,여성_유동인구_수,연령대_10_유동인구_수,연령대_20_유동인구_수,연령대_30_유동인구_수,연령대_40_유동인구_수,연령대_50_유동인구_수,연령대_60_이상_유동인구_수,총_상주인구_수,...,주중_매출_금액,주말_매출_금액,주중_매출_건수,주말_매출_건수,남성_매출_금액,남성_매출_건수,여성_매출_금액,여성_매출_건수,점포수,임대료
0,1078154,556415,521736,63220,353808,235924,168800,157170,99228,1577,...,4.821905e+10,2.646531e+10,2571027,1394990,3.380276e+10,1784205,3.800171e+10,2117802,712,50.2
1,335201,175040,160162,12087,114949,77450,55264,50952,24502,1457,...,4.232019e+09,1.922581e+09,327711,160862,3.102697e+09,262604,2.483960e+09,209923,164,50.2
2,381295,223109,158185,8583,77033,100744,91333,71406,32198,1241,...,4.643661e+09,2.334446e+09,363927,165091,4.002332e+09,327271,2.345138e+09,186702,166,50.2


In [4]:
scaler = preprocessing.MinMaxScaler(feature_range= (0, 1))

In [5]:
x_std = scaler.fit_transform(x_row)
y_std = scaler.fit_transform(y_row)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(x_std, y_std, random_state= seed)

In [7]:
print('X_train.shape : ', X_train.shape, 'y_train.shape : ', y_train.shape)
print('X_test.shape : ', X_test.shape, 'y_test.shape : ', y_test.shape)

X_train.shape :  (4329, 69) y_train.shape :  (4329, 1)
X_test.shape :  (1443, 69) y_test.shape :  (1443, 1)


In [8]:
model = Sequential()
model.add(Dense(512, input_dim= X_train.shape[1], activation= 'relu'))
model.add(Dense(256, activation= 'relu'))
model.add(Dense(256, activation= 'relu'))
model.add(Dense(64, activation= 'relu'))
model.add(Dense(8, activation= 'relu'))
model.add(Dense(1))

model.compile(loss= 'mean_squared_error',
              optimizer= 'adam',
              metrics= ['accuracy'])

early_stopping_callback = EarlyStopping(monitor= 'val_loss', patience= 100)

model.fit(X_train, y_train, validation_split= 0.15,
          epochs= 3000, batch_size= 300, verbose= 0,
          callbacks= [early_stopping_callback])

In [9]:
Y_prediction = model.predict(X_test)

In [10]:
Y_prediction_ = scaler.inverse_transform(Y_prediction)
y_test_ = scaler.inverse_transform(y_test)

In [11]:
for i in range(10):
    label = y_test_[i, 0]
    prediction = Y_prediction_[i, 0]
    print('step :', i, '==> Real : {:6.3f}\t====> pred : {:6.3f}\terror : ====> {:6.3f}'.
          format(label, prediction, label - prediction))

step : 0 ==> Real :  7.300	====> pred :  6.131	error : ====>  1.169
step : 1 ==> Real :  2.100	====> pred :  0.651	error : ====>  1.449
step : 2 ==> Real :  8.600	====> pred :  8.792	error : ====> -0.192
step : 3 ==> Real :  0.500	====> pred :  1.615	error : ====> -1.115
step : 4 ==> Real : 18.500	====> pred : 18.551	error : ====> -0.051
step : 5 ==> Real :  4.700	====> pred :  7.154	error : ====> -2.454
step : 6 ==> Real : 21.600	====> pred : 20.399	error : ====>  1.201
step : 7 ==> Real :  6.000	====> pred :  3.183	error : ====>  2.817
step : 8 ==> Real :  4.500	====> pred :  5.870	error : ====> -1.370
step : 9 ==> Real :  8.900	====> pred :  8.890	error : ====>  0.010


In [12]:
Y_pred = pd.DataFrame(Y_prediction_)
Y_real = pd.DataFrame(y_test_)

In [13]:
result = pd.DataFrame()
result['Y_pred'] = Y_pred.values.flatten()
result['Y_real'] = Y_real.values.flatten()
result['Mean'] = Y_real - Y_pred
result['Mean'] = result['Mean'].abs()
result.describe()

,Y_pred,Y_real,Mean
count,1443.000000,1443.000000,1443.000000
mean,7.693733,8.019335,1.572906
std,4.158426,4.582779,1.640337
min,0.223246,0.200000,0.003892
25%,4.724381,4.600000,0.431659
50%,6.946983,7.300000,1.036172
75%,9.890424,10.950000,2.116720
max,21.723467,24.300000,11.688733
